# Discover the Higgs with Deep Neural Networks
# Chapter 5: Training with Event Weights

In this chapter you will apply event weights for the training. If the weights are applied in the correct way they can significantly improve the training results.

In [ ]:
# Necessary imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy.random import seed
import os

# Import the tensorflow module to create a neural network
import tensorflow as tf
from tensorflow.data import Dataset

# Import function to split data into train and test data
from sklearn.model_selection import train_test_split

# Import some common functions created for this notebook
import common

# Random state
random_state = 21
_ = np.random.RandomState(random_state)

## Data Preparation

### Load the Data

In [ ]:
# Define the input samples
sample_list_signal = ['ggH125_ZZ4lep', 'VBFH125_ZZ4lep', 'WH125_ZZ4lep', 'ZH125_ZZ4lep']
sample_list_background = ['llll', 'Zee', 'Zmumu', 'ttbar_lep']

In [ ]:
sample_path = 'input'
# Read all the samples
no_selection_data_frames = {}
for sample in sample_list_signal + sample_list_background:
    no_selection_data_frames[sample] = pd.read_csv(os.path.join(sample_path, sample + '.csv'))

### Event Pre-Selection

Import the pre-selection functions saved during the first chapter. If the modules are not found solve and execute the notebook of the first chapter.

In [ ]:
from functions.selection_lepton_charge import selection_lepton_charge
from functions.selection_lepton_type import selection_lepton_type

In [ ]:
# Create a copy of the original data frame to investigate later
data_frames = no_selection_data_frames.copy()

# Apply the chosen selection criteria
for sample in sample_list_signal + sample_list_background:
    # Selection on lepton type
    type_selection = np.vectorize(selection_lepton_type)(
        data_frames[sample].lep1_pdgId,
        data_frames[sample].lep2_pdgId,
        data_frames[sample].lep3_pdgId,
        data_frames[sample].lep4_pdgId)
    data_frames[sample] = data_frames[sample][type_selection]

    # Selection on lepton charge
    charge_selection = np.vectorize(selection_lepton_charge)(
        data_frames[sample].lep1_charge,
        data_frames[sample].lep2_charge,
        data_frames[sample].lep3_charge,
        data_frames[sample].lep4_charge)
    data_frames[sample] = data_frames[sample][charge_selection]

### Get Training and Test Data

In [ ]:
# Split data to keep 40% for testing
train_data_frames, test_data_frames = common.split_data_frames(data_frames, 0.6)

## Event Weights

As already explained, significantly more events were simulated than are actually to be expected in the measurement. In addition, the events were generated for different impacts on the final prediction. To account for this, weights are applied to the events to adjust their impact on the prediction. The effect of these event weights can be seen in the following histograms.

<center>Prediction without applying the event weights:</center>
<div>
<img src='figures/event_weights_all_not_applied.png' width='500'/>
</div>


<center>Prediction when applying the event weights:</center>
<div>
<img src='figures/event_weights_all_applied.png' width='500'/>
</div>


The simulated events are multiplied by weight factors for the final prediction of measured data. Thus, events with large weights are more important for the prediction than events with small weights. In order to take this into account for the training, the event weights $w_i$ can be included in the calculation of the loss:<br>
$H = -\frac{1}{N} \sum_i^N w_i (y_i^{true} log(y_i^{predict}) + (1 - y_i^{true}) log(1 - y_i^{predict}))$

In [ ]:
# The training input variables
training_variables = ['lep1_pt', 'lep2_pt', 'lep3_pt', 'lep4_pt']

Extract now also the weights for the tarining and validation data

In [ ]:
# Extract the values, weights, and classification of the data
values, weights, classification = common.get_dnn_input(train_data_frames, training_variables, sample_list_signal, sample_list_background)

In order to also split the weights into a training and validation set the same `train_test_split()` can be used. It is very important that the same split is applied for the weights as for the values and classifications. Otherwise, the weights would no longer be associated with the matching events. This can be achieved by using the same random state for the split of values and classification and the split of the weights.

In [ ]:
# Split into train and validation data
train_values, val_values, train_classification, val_classification = train_test_split(values, classification, test_size=1/3, random_state=random_state)
train_weights, val_weights = train_test_split(weights, classification, test_size=1/3, random_state=random_state)[:2]

## Create the Neural Network

The use of training weights is very straight forward since they can be directly included in the tensorflow datasets

In [ ]:
# Convert the data to tensorflow datasets
train_data = Dataset.from_tensor_slices((train_values, train_classification, train_weights))
train_data = train_data.shuffle(len(train_data), seed=random_state)
train_data = train_data.batch(128)
val_data = Dataset.from_tensor_slices((val_values, val_classification, val_weights))
val_data = val_data.shuffle(len(val_data), seed=random_state)
val_data = val_data.batch(128)

<font color='blue'>
Task:

Let's follow the same strategy as before:
- Recreate and adapt the normalization layer
- Recreate the tensorflow model with 2 hidden layers and 60 nodes per layer
</font>

In [ ]:
# Normalization layer
normalization_layer = tf.keras.layers.Normalization()
normalization_layer.adapt(train_values)
# Create a simple NN
model_layers = [
    normalization_layer,
    tf.keras.layers.Dense(60, activation='relu'),
    tf.keras.layers.Dense(60, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
]
model = tf.keras.models.Sequential(model_layers)

## Train the Neural Network

In [ ]:
# Loss function
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)
# Optimizer
adam_optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0002, beta_1=0.9)

To monitore the training with weights one should use the weighted metric. This metric is calculated with the provided weights of the training dataset.

In [ ]:
# Compile model now with the weighted metric
model.compile(optimizer=adam_optimizer, loss=loss_fn, weighted_metrics=['binary_accuracy'])

In [ ]:
# Early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
# Train model
history = model.fit(train_data, validation_data=val_data, callbacks=[early_stopping], epochs=1000)

In [ ]:
# Plot the training history
fig, ax = plt.subplots(figsize=(7, 6))
ax.plot(history.history['loss'], label='training')
ax.plot(history.history['val_loss'], label='validation')
ax.set_xlabel('epoch')
ax.set_ylabel('loss')
ax.legend()
_ = plt.show()

<font color='blue'>
Task:

Explain the difference in the training loss when the weights are applied.
</font>

<font color='green'>
Answer:

For the calculation of the loss the weights are applied for each event. Since the mean training weight is $4.45 \cdot 10^{-4}$ the loss is reduced by the same magnitude.
</font>

In [ ]:
print(f'The mean training weight is {np.round(train_weights.mean(), 6)}')

## Apply and Evaluate the Neural Network on Training and Validation Data

<font color='blue'>
Task:

- Apply the model on the training and validation data and plot the classification
- Evaluate the loss and accuracy for the training and validation data
</font>

In [ ]:
# Apply the model for training and validation values
train_prediction = model.predict(train_values)
val_prediction = model.predict(val_values)

# Plot the model output
common.plot_dnn_output(train_prediction, train_classification, val_prediction, val_classification)
_ = plt.show()

In [ ]:
# Evaluate the model on training and validation data
model_train_evaluation = model.evaluate(train_data)
model_val_evaluation = model.evaluate(val_data)

print(f'train loss = {round(model_train_evaluation[0], 5)}\ttrain binary accuracy = {round(model_train_evaluation[1], 5)}')
print(f'val loss = {round(model_val_evaluation[0], 5)}\tval binary accuracy = {round(model_val_evaluation[1], 5)}')

<font color='blue'>
Task:

What differences can you see on the accuarcy and classification distribution? Explain these differences.
</font>

<font color='green'>
Answer:

The accuracy has significantly improved to 97.5% during the whole training. The reason for this can be seen in the classification plots. Since the weights of signal events are much smaller the model focuses mostly on the classification of background events during the training and therefore classifies all events close to background. With a signal purity of about 2.5% this leads to a accuracy of 97.5%.
</font>

In [ ]:
# Check the event weights
mean_signal_weight = weights[classification > 0.5].mean()
mean_background_weight = weights[classification < 0.5].mean()
print(f'The mean weight for signal events is {np.round(mean_signal_weight, 7)} and for background events {np.round(mean_background_weight, 7)}')

## Reweight the Event Weights

Introducing the event weights caused the problem that the overall background dominates the training. With the event weights of the simulated data the loss is already quite low if all data is consequently classified close to background.<br>
$H = -\frac{1}{N} \sum_i^N w_i (y_i^{true} log(y_i^{predict}) + (1 - y_i^{true}) log(1 - y_i^{predict}))$

To solve this problem we should reweight the weights to have the same effect on training for signal and background events.

<font color='blue'>
Task:

Write a function that reweights the given weights:<br>
1. Take the absolute value of the weight to not run into problems with negative weights
2. Split the weights into signal and background weights (by setting the other weights in the array to zero)
3. Scale the signal weights to have the total sum as the background weights
4. Merge the background and scaled signal weights
5. Scale all weights to have a mean weight of 1
</font><br>

In [ ]:
def reweight_weights(weights, classification):
    # Take the absolute value of the weight
    weights_abs = np.abs(weights)
    # Split in signal and background weights
    weights_signal = weights_abs*classification
    weights_background = weights_abs*(1 - classification)
    # Scale the signal events
    weights_signal_scaled = weights_signal * sum(weights_background) / sum(weights_signal)
    # Merge the signal and background events
    weights_reweighted = weights_background + weights_signal_scaled
    # Normalize mean weight to one
    weights_reweighted /= weights_reweighted.mean()
    return weights_reweighted

<font color='blue'>
Task:

Test your reweighting function. The mean of all weights should be 1 and the sum of signal weights and background weights should be equal.
</font>

In [ ]:
weights_reweighted = reweight_weights(weights, classification)
signal_weights_reweighted = weights_reweighted[classification > 0.5]
background_weights_reweighted = weights_reweighted[classification < 0.5]

print(f'Mean weight: {weights_reweighted.mean()}')
print(f'Signal weight sum: {signal_weights_reweighted.sum()}')
print(f'Background weight sum: {background_weights_reweighted.sum()}')

When you are happy with your reweighting function save it for the use in the next chapters.<br>
If you have used any imported modules first save these imports to the file.

In [ ]:
%%writefile functions/reweight_weights.py
import numpy as np

In [ ]:
from inspect import getsource, getmodulename
%save -a functions/reweight_weights.py getsource(reweight_weights)

## Recreate and Retrain the Neural Network

<font color='blue'>
Task:

Create tensorflow datasets with the reweighted event weights for training and validation.
</font>

In [ ]:
# Get reweighted weights
train_weights_reweighted = reweight_weights(train_weights, train_classification)
val_weights_reweighted = reweight_weights(val_weights, val_classification)

# Convert the data to tensorflow datasets
train_data = Dataset.from_tensor_slices((train_values, train_classification, train_weights_reweighted))
train_data = train_data.shuffle(len(train_data), seed=random_state)
train_data = train_data.batch(128)
val_data = Dataset.from_tensor_slices((val_values, val_classification, val_weights_reweighted))
val_data = val_data.shuffle(len(val_data), seed=random_state)
val_data = val_data.batch(128)

In [ ]:
# Normalization layer
normalization_layer = tf.keras.layers.Normalization()
normalization_layer.adapt(train_values)
# Create a simple NN
model_layers = [
    normalization_layer,
    tf.keras.layers.Dense(60, activation='relu'),
    tf.keras.layers.Dense(60, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
]
model = tf.keras.models.Sequential(model_layers)

In [ ]:
# Compile model
model.compile(optimizer=adam_optimizer, loss=loss_fn, weighted_metrics=['binary_accuracy'])
# Train model
history = model.fit(train_data, validation_data=val_data, callbacks=[early_stopping], epochs=1000)

In [ ]:
# Plot the training history
fig, ax = plt.subplots(figsize=(7, 6))
ax.plot(history.history['loss'], label='training')
ax.plot(history.history['val_loss'], label='validation')
ax.set_xlabel('epoch')
ax.set_ylabel('loss')
ax.legend()
_ = plt.show()

Since you reweighted the mean training and validation weight to one the loss is in the same magnetude as in the chapters before. 

## Apply and Evaluate the Neural Network on Training and Validation Data

In [ ]:
# Apply the model for training and validation values
train_prediction = model.predict(train_values)
val_prediction = model.predict(val_values)
# Plot the model output
common.plot_dnn_output(train_prediction, train_classification, val_prediction, val_classification)
_ = plt.show()

In [ ]:
# Evaluate the model on training and validation data
model_train_evaluation = model.evaluate(train_data)
model_val_evaluation = model.evaluate(val_data)

print(f'train loss = {round(model_train_evaluation[0], 5)}\ttrain binary accuracy = {round(model_train_evaluation[1], 5)}')
print(f'val loss = {round(model_val_evaluation[0], 5)}\tval binary accuracy = {round(model_val_evaluation[1], 5)}')

## Prediction on Test Data

In [ ]:
# Apply the model on the test data
data_frames_apply_dnn = common.apply_dnn_model(model, test_data_frames, training_variables, sample_list_signal + sample_list_background)
model_prediction = {'variable': 'model_prediction',
                    'binning': np.linspace(0, 1, 50),
                    'xlabel': 'prediction'}
common.plot_hist(model_prediction, data_frames_apply_dnn, show_data=False)
plt.show()

<font color='blue'>
Task:

Compare this classification plot with the one resulting from the training without event weights in chapter 2.
</font>

<font color='green'>
Answer:

The overall classification of signal and $llll$ background events looks quite similar to the distribution in chapter 2. However, the classification of backgrounds modelled by a low number of generated events has significantly improved.
</font>

## Save and Load a Model

Save the model for a later comparison

In [ ]:
model.save('models/chapter5_model')